# OTC Markets
We wraped the data from [OTC Markets.](https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume) Then, we were fail to wrap all of data (18524 companies) by using splinter "click_link" because the MORE button design of this website is a Javescript which would be triggered when user click it. Therefore, we decided to click MORE button by ourselves to show all of data and wrap it.

In [1]:
from splinter import Browser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
import pymysql

In [22]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"
browser.visit(url)

In [3]:
#We click MORE button for several times to get all of data.
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [4]:
#Wrap the whole table content.
OTC_market_soup = soup.find('table', class_='_1DoSj9nrVo')
#Change bs4 object into html format
html_content = OTC_market_soup.prettify()

In [5]:
#Using Pandas to read html
OTC_market_list = pd.read_html(html_content)

In [7]:
#Convert it to DataFrame and rename columns name
OTC_market_table = OTC_market_list[0]
OTC_market_table.rename(columns={"Symbol  Sym":"company","Price  Price":"price",
                                 "% Change  % Change": "change_percent", "$ Vol  $ Vol": "vol",
                                 "Share Vol  Share Vol":"share_vol","Trades  Trades":"trades"},inplace=True)
OTC_market_table.head(10)

,company,price,change_percent,vol,share_vol,trades
0,CSGKF,13.3100,0.23,2938102,222533,50
1,CURLF,10.7800,8.78,1990604,186077,494
2,TCEHY,49.4100,0.35,1812836,36682,208
3,GLCNF,3.9966,-0.16,1799338,450217,6
4,CMXHF,140.9500,0.00,1592338,11292,3
5,OGRMF,7.4900,0.54,1484344,196929,506
6,SFTBY,53.2600,0.15,1373286,25790,60
7,CSLLY,70.6800,-0.92,860877,12211,6
8,GBTC,6.7100,1.51,731449,109399,117
9,RHHBY,32.9360,-0.38,716182,21690,60


In [8]:
#Wrap the icon image link from website in order to identify which Symbol it is.
#Find all of icon image link
icon_list = OTC_market_soup.find_all("div",class_="_2_IqcDGekT")
symbol_icon = []
for x in icon_list:
    #Only take symbol portion from link
    symbol_icon.append(x.img["src"][12:14])

In [9]:
#Combine symbol data into OTC Market
OTC_market_table["symbol"] = symbol_icon
OTC_market_table.head()

,company,price,change_percent,vol,share_vol,trades,symbol
0,CSGKF,13.3100,0.23,2938102,222533,50,ps
1,CURLF,10.7800,8.78,1990604,186077,494,qx
2,TCEHY,49.4100,0.35,1812836,36682,208,ps
3,GLCNF,3.9966,-0.16,1799338,450217,6,ps
4,CMXHF,140.9500,0.00,1592338,11292,3,ps


We only chose QX and QB symbol.

In [10]:
OTC_QX_QB = OTC_market_table.loc[(OTC_market_table.symbol == "qx") | (OTC_market_table.symbol == "qb"),:]
OTC_QX_QB.reset_index(drop=True,inplace=True)
OTC_QX_QB.head(10)

,company,price,change_percent,vol,share_vol,trades,symbol
0,CURLF,10.7800,8.78,1990604,186077,494,qx
1,OGRMF,7.4900,0.54,1484344,196929,506,qx
2,GBTC,6.7100,1.51,731449,109399,117,qx
3,RHHBY,32.9360,-0.38,716182,21690,60,qx
4,ACRGF,22.6113,1.58,497366,22053,60,qx
5,TGODF,3.4100,1.79,448388,131052,127,qx
6,GTBIF,14.9706,0.39,367509,24613,105,qx
7,CVSI,5.2100,-1.14,365058,69336,103,qb
8,VGWCF,3.2500,-3.56,309490,93623,144,qb
9,AVMXY,4.9300,-6.10,294467,59678,67,qx


In [103]:
OTC_QX_QB.to_csv("part_qx_qb.csv")

## Connect to local database

In [13]:
rds_connection_string = "root:Pega#123456@127.0.0.1/ETL_Market"
engine = create_engine(f'mysql://{rds_connection_string}')

In [14]:
#Check for tables
engine.table_names()

['OTC_company']

In [16]:
#Use soup to scrape web, then use pandas converted DataFrame into database
OTC_QX_QB.to_sql(name='OTC_company', con=engine, if_exists='append', index=False)

In [17]:
#Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from OTC_company', con=engine)

,id,company,price,change_percent,vol,share_vol,trades,symbol
0,1,CURLF,10.7800,8.78,1990604,186077,494,qx
1,2,OGRMF,7.4900,0.54,1484344,196929,506,qx
2,3,GBTC,6.7100,1.51,731449,109399,117,qx
3,4,RHHBY,32.9360,-0.38,716182,21690,60,qx
4,5,ACRGF,22.6113,1.58,497366,22053,60,qx
5,6,TGODF,3.4100,1.79,448388,131052,127,qx
6,7,GTBIF,14.9706,0.39,367509,24613,105,qx
7,8,CVSI,5.2100,-1.14,365058,69336,103,qb
8,9,VGWCF,3.2500,-3.56,309490,93623,144,qb
9,10,AVMXY,4.9300,-6.10,294467,59678,67,qx


In [1]:
"""
for x in range(2):
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    
    #divs = browser.find_by_tag("div")
    #browser.click_link_by_partial_text("yi4dEwAzKE")
    #browser.find_link_by_text('yi4dEwAzKE').first.click()
    #browser.find_by_text('MORE').first.click()
    #divs.first.find_by_tag("MORE").first.click()
    
    browser.execute_script("$('body').empty()")
"""

'\nfor x in range(2):\n    html = browser.html\n    soup = BeautifulSoup(html,\'html.parser\')\n    \n    #divs = browser.find_by_tag("div")\n    #browser.click_link_by_partial_text("yi4dEwAzKE")\n    #browser.find_link_by_text(\'yi4dEwAzKE\').first.click()\n    #browser.find_by_text(\'MORE\').first.click()\n    #divs.first.find_by_tag("MORE").first.click()\n    \n    browser.execute_script("$(\'body\').empty()")\n'